In [1]:
import requests
import urllib.parse as parse
import csv

import os
from dotenv import load_dotenv
load_dotenv()


API_KEY = os.environ['API_KEY']
COMMENT_URL = os.environ['COMMENT_URL']
URL_HEAD = COMMENT_URL + '?'


import src.video_id as vid
import src.channel_id as cid

nextPageToken = ''
item_count = 0
items_output = [
    ['videoId']+
    ['textDisplay']+
    ['textOriginal']+
    ['authorDisplayName']+
    ['authorProfileImageUrl']+
    ['authorChannelUrl']+
    ['authorChannelId']+
    ['canRate']+
    ['viewerRating']+
    ['likeCount']+
    ['publishedAt']+
    ['updatedAt']
]

In [2]:
#パラメータ設定
video_id = vid.VIDEO_ID[0]
channelId = cid.CHANNEL_ID[0]
# 1リクエスト(exe_num=1)で100件のコメントを取得できるようになっている
exe_num = 10

In [3]:
for i in range(exe_num):

    #APIパラメータセット
    param = {
        'key':API_KEY,
        'part':'snippet',
        #----↓フィルタ（いずれか1つ）↓-------
        # 'allThreadsRelatedToChannelId':channelId,
        'videoId':video_id,
        #----↑フィルタ（いずれか1つ）↑-------
        'maxResults':'100',
        'moderationStatus':'published',
        'order':'relevance',
        'pageToken':nextPageToken,
        'searchTerms':'',
        'textFormat':'plainText',
    }
    #リクエストURL作成
    target_url = URL_HEAD + (parse.urlencode(param))

    #データ取得
    res = requests.get(target_url).json()

    #件数
    item_count += len(res['items'])

    #print(target_url)
    print(str(item_count)+"件")

    #コメント情報を変数に格納
    for item in res['items']:
        items_output.append(
            [str(item['snippet']['topLevelComment']['snippet']['videoId'])]+
            [str(item['snippet']['topLevelComment']['snippet']['textDisplay'].replace('\n', ''))]+
            [str(item['snippet']['topLevelComment']['snippet']['textOriginal'])]+
            [str(item['snippet']['topLevelComment']['snippet']['authorDisplayName'])]+
            [str(item['snippet']['topLevelComment']['snippet']['authorProfileImageUrl'])]+
            [str(item['snippet']['topLevelComment']['snippet']['authorChannelUrl'])]+
            [str(item['snippet']['topLevelComment']['snippet']['authorChannelId']['value'])]+
            [str(item['snippet']['topLevelComment']['snippet']['canRate'])]+
            [str(item['snippet']['topLevelComment']['snippet']['viewerRating'])]+
            [str(item['snippet']['topLevelComment']['snippet']['likeCount'])]+
            [str(item['snippet']['topLevelComment']['snippet']['publishedAt'])]+
            [str(item['snippet']['topLevelComment']['snippet']['updatedAt'])]
        )

    #nextPageTokenがなくなったら処理ストップ
    if 'nextPageToken' in res:
        nextPageToken = res['nextPageToken']
    else:
        break

100件
200件
300件
400件
500件
600件
700件
800件
900件
1000件


In [4]:
#CSVで出力
f = open('youtube-comments-list.csv', 'w', newline='', encoding='UTF-8')
writer = csv.writer(f)
writer.writerows(items_output)
f.close()